In [1]:
# Dictate the environment's loctaion of REFPROP
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

In [2]:
# Import the main class from the Python library
from ctREFPROP.ctREFPROP import REFPROPFunctionLibrary

# Imports from conda-installable packages
import pandas as pd

# Import numpy
import numpy as np

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import Math for common values such as PI
import math


In [3]:
# Instantiate the library, and use the environment variable to explicitly state which path we want to use.
# As mentioned above, this will be changed to call the correct REFPROP functions to be used
# with MINI-REFPROP and 32-bit python.
# If using MINI-REFPROP and 32-bit python please make the following changes
# RP = REFPROPFunctionLibrary('C:/Program FIles (x86)/MINI-REFPROP\\REFPROP.DLL')
RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])

In [4]:
# This will call which root directory that will be used for the program. 
RP.SETPATHdll(os.environ['RPPREFIX'])

In [5]:
# Get the unit system we want to use (Mass base SI gives units in
# K, Pa, kg, m, N, J, W, and s)
MASS_BASE_SI = RP.GETENUMdll(0, "MASS BASE SI").iEnum

#### sCO2 States

In [6]:
# Hold one temperature constant and vary the other 
# Specify Compressor Inlet temperature constant (max heat dissipation is 15 kW)
# Specify Engine Inlet temperature (max heat input is 16 kW)

In [7]:
m_dot = .2 # kg/s of mass flow rate of sCO2

# Outlet of Heat Source / Inlet of Engine
T3 = 90 # [C]
T3 = T3 + 273.15
P3 = 2500 # [psia]
P3 = P3 * 6894.8 # convert psia to Pa

T1 = 39 # [C]
T1 = T1 + 273.15

In [8]:
# STATE 3 PROPERTIES
State_3 = RP.REFPROPdll("CO2","PT","H;D;S;CP/CV;", MASS_BASE_SI,0,0,P3,T3,[1.0])
State_3 = pd.DataFrame(State_3.Output[0:4],
            index = ['Enthalpy [J/kg]','Density [kg/m^3]','Entropy [J/kg]','CP/CV'],
            columns = ['State 3'])

# OUTLET OF ENGINE
mass_cylinder = State_3.loc['Density [kg/m^3]', 'State 3'] * .000308276
State_4_den = mass_cylinder / .000454574
# With Isentropic expansion
State_4_entr = State_3.loc['Entropy [J/kg]', 'State 3']

State_4_isen = RP.REFPROPdll("CO2","DS","P;T;H;CP/CV", MASS_BASE_SI,0,0,State_4_den,State_4_entr,[1.0])
State_4_isen = pd.DataFrame(State_4_isen.Output[0:4],
            index = ['Pressure [Pa]', 'Temperature [K]', 'Enthalpy [J/kg]','CP/CV'],
            columns = ['State 4 Isentropic'])

# Outlet of Heat Exchanger
P1 = State_4_isen.loc['Pressure [Pa]', 'State 4 Isentropic']
State_1 = RP.REFPROPdll("CO2","PT","H;D;S;CP/CV;", MASS_BASE_SI,0,0,P1,T1,[1.0])
State_1 = pd.DataFrame(State_1.Output[0:4],
            index = ['Enthalpy [J/kg]','Density [kg/m^3]','Entropy [J/kg]','CP/CV'],
            columns = ['State 1'])

# Outlet of Compressor
State_2_isen = RP.REFPROPdll("CO2","PS","T;D;H;CP/CV", MASS_BASE_SI,0,0,P3,State_1.loc['Entropy [J/kg]', 'State 1'],[1.0])
State_2_isen = pd.DataFrame(State_2_isen.Output[0:4],
            index = ['Temperature [K]','Density [kg/m^3]', 'Enthalpy [J/kg]','CP/CV'],
            columns = ['State 2 Isentropic'])

In [9]:
# Fill missing Data points
State_3.loc['Pressure [Pa]', 'State 3'] = P3
State_3.loc['Temperature [K]', 'State 3'] = T3
State_4_isen.loc['Density [kg/m^3]', 'State 4 Isentropic'] = State_4_den
State_4_isen.loc['Entropy [J/kg]', 'State 4 Isentropic'] = State_4_entr
State_1.loc['Pressure [Pa]', 'State 1'] = P1
State_1.loc['Temperature [K]', 'State 1'] = T1
State_2_isen.loc['Entropy [J/kg]', 'State 2 Isentropic'] = State_1.loc['Entropy [J/kg]', 'State 1']
State_2_isen.loc['Pressure [Pa]', 'State 2 Isentropic'] = P3

# Combine the data frames into one data frame for ease of use
sCO2_States = pd.concat([State_1, State_2_isen, State_3, State_4_isen], axis =1)
# Reorder the Data Frame
sCO2_States = sCO2_States.reindex(["Pressure [Pa]", "Temperature [K]", 'Density [kg/m^3]', 'Enthalpy [J/kg]',
                     'Entropy [J/kg]', 'CP/CV'])
sCO2_States

,State 1,State 2 Isentropic,State 3,State 4 Isentropic
Pressure [Pa],8.478766e+06,1.723700e+07,1.723700e+07,8.478766e+06
Temperature [K],3.121500e+02,3.517691e+02,3.631500e+02,3.156501e+02
Density [kg/m^3],3.793996e+02,5.239983e+02,4.509085e+02,3.057902e+02
Enthalpy [J/kg],3.696321e+05,3.887446e+05,4.199911e+05,3.970620e+05
Entropy [J/kg],1.546580e+03,1.546580e+03,1.634027e+03,1.634027e+03
CP/CV,1.018872e+01,3.117937e+00,2.821532e+00,5.297056e+00


In [10]:
Q_in = m_dot * (sCO2_States.loc['Enthalpy [J/kg]', 'State 3'] - sCO2_States.loc['Enthalpy [J/kg]', 'State 2 Isentropic'])
Q_out = m_dot * (sCO2_States.loc['Enthalpy [J/kg]', 'State 4 Isentropic'] - sCO2_States.loc['Enthalpy [J/kg]', 'State 1'])
Isen_Work_in = m_dot * (sCO2_States.loc['Enthalpy [J/kg]', 'State 2 Isentropic'] - sCO2_States.loc['Enthalpy [J/kg]', 'State 1'])
Isen_Work_out = m_dot * (sCO2_States.loc['Enthalpy [J/kg]', 'State 3'] - sCO2_States.loc['Enthalpy [J/kg]', 'State 4 Isentropic'])

print("Inlet Compressor Temp =", T1 - 273.15, "C")
print("Inlet Engine Temp =", T3 - 273.15, "C")
print("Heat into Cycle =", round(Q_in/1000,3), "kW")
print("Heat out of Cycle =", round(Q_out/1000,3), "kW")
print("Isen Work by Engine =", round(Isen_Work_out/1000,3), "kW")
print("Isen Therm. Eff =", round((Isen_Work_out - Isen_Work_in)/(Q_in) * 100,3), "%")
print("Isen Work by Compressor =", round(Isen_Work_in/1000,3), "kW")
print("Inlet Compressor Pressure=", round(sCO2_States.loc['Pressure [Pa]', 'State 1'] / 6894.8 , 3), "psi" )

Inlet Compressor Temp = 39.0 C
Inlet Engine Temp = 90.0 C
Heat into Cycle = 6.249 kW
Heat out of Cycle = 5.486 kW
Isen Work by Engine = 4.586 kW
Isen Therm. Eff = 12.214 %
Isen Work by Compressor = 3.823 kW
Inlet Compressor Pressure= 1229.733 psi


In [11]:
print(T1 - 273.15 , T3 - 273.15, round(Q_in/1000,3), round(Q_out/1000,3), round(Isen_Work_out/1000,3), round((Isen_Work_out - Isen_Work_in)/(Q_in) * 100,3), round(Isen_Work_in/1000,3), round(sCO2_States.loc['Pressure [Pa]', 'State 1'] / 6894.8 , 3))

39.0 90.0 6.249 5.486 4.586 12.214 3.823 1229.733
